In [1]:
import pandas as pd
import ahocorasick
import date_fixer
%load_ext autoreload

%autoreload 2

In [ ]:
df = pd.read_csv('/Users/kylegreen/codeup-data-science/trials_in_rust/trials-in-rust/first_rust_program/Testing/Almustaqbal.csv')
df.head()

In [ ]:
df.head(1).headline.values

In [ ]:
df.head(1).text

In [ ]:
df.head(1).text.values

In [2]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline

#mix = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-mix-sentiment")
#ca = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-ca-sentiment")
#da = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-da-sentiment")
msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")

#half = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-half")
#quarter = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-quarter")
#eighth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-eighth")
#sixteenth = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sixteenth")


In [ ]:

models = (mix, ca, da, msa, half, quarter, eighth, sixteenth)

In [ ]:
for mod in models:
    print(mod('hello'))

In [ ]:
df.head(5).text.values

In [ ]:
sum([len(val.split()) for val in df.head(100).text.values])/100
modelable = df.head(2000).text[[True if len(val.split()) < 350 else False for val in df.head(2000).text.values]]
for val in modelable.values:
    print(msa(val))


In [ ]:
for val in df.head(200).values:
    try:
        print(msa(val))
    except:
        continue
    
    #for mod in models:
        #print(mod(val[:512]))
        
subset = df.head(100).copy()

        
        
subset['label'] = subset.apply(lambda x:  msa(x.text)[0]['label'] if len(x.text.split()) < 300 else False, axis=1)
subset['score'] = subset.apply(lambda x: msa(x.text)[0]['score'] if len(x.text.split()) < 300 else False, axis=1)
subset.head()

#texts = {}    
#for i, val in enumerate(df.head(200).text.values):
#    if len(val) > 35 and len(val) < 512:
#        texts[i] = val
        
        
#print(texts)
        
        

In [ ]:
subset.head(15)

In [ ]:
subset.score.groupby(subset.label).max()

In [ ]:
subset[subset.label== 'negative'].max()

In [ ]:
translator = Translator()

In [ ]:
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

translate_to_english(subset.head(1).text)

In [ ]:
subset.to_csv('labels.csv', index=False)

In [ ]:
sub = pd.read_csv('labels.csv')
sub.head()

In [ ]:
translator = Translator()
def translate_to_english(text):
    translation = translator.translate(text, dest='en').text
    return translation

def check_for_america(translation):
    if 'america' in translation.lower() or 'united states' in translation.lower():
        return True
    else:
        return False

text = sub.head(1).text



#translation = translate_to_english(text)
#check_for_america(translation)

row_nums = df.shape[0]
sets_of_ten = int(row_nums / 100)

collection_of_dfs = []
print(sets_of_ten)

for i in range(0,100):
    collection_of_dfs.append(df[df.index.isin(range(i * sets_of_ten, (i+1) * sets_of_ten))].copy())
    collection_of_dfs[i]['america'] = [True if val.str.contains('')]
    print(collection_of_dfs[f'frame_{i}'])
    break
    

#sub['en_text'] = sub.apply(lambda x: translate_to_english(x.text), axis=1)
#sub.head(10)

In [ ]:
df['source'] = 'almustaqbal'

In [2]:
import pandas as pd
import ahocorasick



In [8]:


def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

def using_ahocorasick(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    tags = col.apply(lambda x: list(A.iter(x)))
    return mask, tags

def look_for_words_in_text(df_text):
    
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}
    
    topics = flip_key_value_pairs(topics)
    
    tags = []
    for key in topics.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

def make_relevant_tagged_df(df):
    topics = {'America' : 'أمريكا',
            'American' : 'أمريكيّ',
            'American (f)' : 'أمريكيّة',
            'American (pl)' : 'أمريكيّين',
            'The United States' : 'الولايات المتحدة',
            'The United States' : 'دول موحّدة',
            'Washington' : 'واشنطن',
            'Bush' : 'بوش',
            'Obama' : 'أوباما',
            'Cheney' : 'تشيني',
            'Clinton' : 'كلينتون',
            'Osama Bin Laden' : 'أسامة بن لادن',
            'Al Gore' : 'آل غور',
            'World Trade Center' : 'مركز التجارة العالمي',
            '9/11' : '9/11',
            'September 11' : '11 سبتمبر',
            'Gulf War' : 'حرب الخليج',
            'Google' : 'غوغل',
            'Facebook' : 'فيسبوك',
            'Al Qaida' : 'القاعدة'}


    topics = flip_key_value_pairs(topics)
    mask, tags = using_ahocorasick(df.text, list(topics.keys()))

    copied = df[mask].copy()

    return copied, tags



Alittihad being processed
0         []
1         []
2         []
3         []
4         []
          ..
318935    []
318936    []
318937    []
318938    []
318939    []
Name: text, Length: 318940, dtype: object
labels and source columns created
Alittihad
Almasryalyoum being processed
0         []
1         []
2         []
3         []
4         []
          ..
261206    []
261207    []
261208    []
261209    []
261210    []
Name: text, Length: 261211, dtype: object
labels and source columns created
Almasryalyoum
Almustaqbal being processed
0         []
1         []
2         []
3         []
4         []
          ..
403241    []
403242    []
403243    []
403244    []
403245    []
Name: text, Length: 403246, dtype: object
labels and source columns created
Almustaqbal
Alqabas being processed
0         []
1         []
2         []
3         []
4         []
          ..
355830    []
355831    []
355832    []
355833    []
355834    []
Name: text, Length: 355835, dtype: object
labels and sou

In [6]:
pd.read_csv('non_tagged_Sabanews.csv')

,id,url,headline,dateline,text,source
0,SBN_ARB_0000002,http://www.sabanews.net/ar/news200066.htm,مناقشة إستراتيجية التواصل الخاصة بالأجندة الوط...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء ـ سبأنت: جرى اليوم بوزار...,Sabanews
1,SBN_ARB_0000003,http://www.sabanews.net/ar/news200073.htm,نائب وزير التخطيط يلتقي بعثة الوكالة اليابانية...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء ـ سبأنت: أشاد نائب وزير ...,Sabanews
2,SBN_ARB_0000004,http://www.sabanews.net/ar/news200090.htm,الناطق الرسمي: الرؤية الحكيمة للدولة في التعام...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء – سبأنت : أكد الناطق الر...,Sabanews
3,SBN_ARB_0000005,http://www.sabanews.net/ar/news200092.htm,نوكيا تطرح خدمة الخرائط والملاحة في الأسواق,08/ديسمبر/2009,08/ديسمبر/2009] القاهرة ـ سبأنت: عبداللطيف الك...,Sabanews
4,SBN_ARB_0000006,http://www.sabanews.net/ar/news200097.htm,الجزائية الاستئنافية بالعاصمة ترفض استئناف الم...,08/ديسمبر/2009,08/ديسمبر/2009] صنعاء – سبأنت: رفضت الشعبة الا...,Sabanews


In [26]:
source_ratios = pd.read_csv('all_articles.csv').source.value_counts(normalize=True).to_dict()
source_ratios

for name, pct in source_ratios.items():
    length = pd.read_csv('all_articles.csv').shape[0]
    df = pd.read_csv('non_tagged_'+name+'.csv').sample(int(pct * length))
    df.to_csv(f'sampled_non_tagged_{name}.csv', index=False)
        


In [27]:
import os
my_sampled_non_tagged_dfs = []
for file in os.listdir():
    if 'sampled_non' in file:
        my_sampled_non_tagged_dfs.append(pd.read_csv(file))
        
total_sampled_non_tagged = pd.concat(my_sampled_non_tagged_dfs)
print(total_sampled_non_tagged.sample(15))



                    id                                                url  \
2986   SHG_ARB_0029910  http://www.echoroukonline.com/ara/articles/378...   
29309  ETD_ARB_0180540  http://www.alittihad.ae/details.php?id=36968&y...   
72206  YM7_ARB_0934648       http://www1.youm7.com/News.asp?NewsID=971685   
8301   YM7_ARB_0041996        http://www1.youm7.com/News.asp?NewsID=59470   
23995  TRN_ARB_0089046  http://tishreen.news.sy/tishreen/public/read/8...   
36865  MTL_ARB_0291664  http://www.almustaqbal.com/v4/article.aspx?typ...   
17020  YM7_ARB_0577331       http://www1.youm7.com/News.asp?NewsID=607470   
9740   YM7_ARB_0230629       http://www1.youm7.com/News.asp?NewsID=252824   
9898   YM7_ARB_0293096       http://www1.youm7.com/News.asp?NewsID=316432   
27152  TRN_ARB_0276071  http://tishreen.news.sy/tishreen/public/read/2...   
3393   SYM_ARB_0400547  http://www.alyaum.com/oldsite/issue/article.ph...   
19266  MTL_ARB_0285659  http://www.almustaqbal.com/v4/article.aspx?typ...   

In [28]:
total_sampled_non_tagged.source.value_counts(normalize=True)

Youm7             0.225606
SaudiYoum         0.151853
Almustaqbal       0.119626
Ryiadh            0.101664
Techreen          0.094436
Almasryalyoum     0.085282
Alittihad         0.084977
Alqabas           0.078258
Echoroukonline    0.042684
Sabanews          0.015614
Name: source, dtype: float64

In [31]:
total_sampled_non_tagged.sample(100000).source.value_counts(normalize=True)

Youm7             0.22596
SaudiYoum         0.15270
Almustaqbal       0.11853
Ryiadh            0.10109
Techreen          0.09385
Alittihad         0.08558
Almasryalyoum     0.08511
Alqabas           0.07811
Echoroukonline    0.04346
Sabanews          0.01561
Name: source, dtype: float64

In [32]:
our_sample = total_sampled_non_tagged.sample(100000).reset_index()
our_sample.to_csv('total_non_tagged_sample.csv', index=False)

In [38]:
date_fixer.make_datetime(our_sample).to_csv('total_non_tagged_sample.csv', index=False)

YM7_ARB_0198899 successful
YM7_ARB_0488162 successful
YM7_ARB_0087860 successful
YM7_ARB_0270426 successful
QBS_ARB_0198555 successful
ETD_ARB_0012424 successful
MTL_ARB_0302461 successful
MSY_ARB_0259191 successful
SYM_ARB_0208718 successful
YM7_ARB_0621243 successful
YM7_ARB_0426048 successful
SYM_ARB_0430452 successful
QBS_ARB_0141043 successful
SYM_ARB_0586480 successful
QBS_ARB_0253585 successful
SYM_ARB_0321525 successful
QBS_ARB_0023382 successful
SYM_ARB_0022153 successful
TRN_ARB_0299148 successful
RYD_ARB_0112935 successful
SYM_ARB_0200199 successful
SYM_ARB_0750412 successful
YM7_ARB_0543874 successful
TRN_ARB_0070352 successful
MTL_ARB_0357627 successful
SYM_ARB_0627399 successful
QBS_ARB_0790481 successful
MTL_ARB_0194745 successful
YM7_ARB_0880997 successful
ETD_ARB_0318069 successful
RYD_ARB_0144921 successful
YM7_ARB_0837958 successful
SHG_ARB_0104470 successful
MTL_ARB_0359075 successful
YM7_ARB_0792171 successful
MSY_ARB_0144385 successful
SYM_ARB_0741336 successful
E

QBS_ARB_0773842 successful
ETD_ARB_0097253 successful
MSY_ARB_0028462 successful
YM7_ARB_0475318 successful
MSY_ARB_0059311 successful
SYM_ARB_0590776 successful
QBS_ARB_0000533 successful
MTL_ARB_0036139 successful
MSY_ARB_0087244 successful
SHG_ARB_0097712 successful
TRN_ARB_0175737 successful
RYD_ARB_0743115 successful
SYM_ARB_0689673 successful
SHG_ARB_0034342 successful
TRN_ARB_0239148 successful
MSY_ARB_0111689 successful
QBS_ARB_0053586 successful
MSY_ARB_0281506 successful
SYM_ARB_0621748 successful
YM7_ARB_0067853 successful
YM7_ARB_0543636 successful
YM7_ARB_0269623 successful
SHG_ARB_0126548 successful
SYM_ARB_0835749 successful
SYM_ARB_0445076 successful
RYD_ARB_0852549 successful
TRN_ARB_0184385 successful
SYM_ARB_0002267 successful
RYD_ARB_0760306 successful
TRN_ARB_0106949 successful
SYM_ARB_0297457 successful
YM7_ARB_0915826 successful
QBS_ARB_0023901 successful
MTL_ARB_0241333 successful
SHG_ARB_0070529 successful
MTL_ARB_0177740 successful
YM7_ARB_0771363 successful
E

QBS_ARB_0230962 successful
SYM_ARB_0651889 successful
TRN_ARB_0246915 successful
RYD_ARB_0372794 successful
SHG_ARB_0135410 successful
YM7_ARB_1008329 successful
SYM_ARB_0467306 successful
ETD_ARB_0257586 successful
QBS_ARB_0270256 successful
YM7_ARB_0192898 successful
MSY_ARB_0012183 successful
MTL_ARB_0055511 successful
YM7_ARB_0317521 successful
MTL_ARB_0030064 successful
TRN_ARB_0048522 successful
QBS_ARB_0044876 successful
MTL_ARB_0054062 successful
ETD_ARB_0285935 successful
MTL_ARB_0001613 successful
MTL_ARB_0297548 successful
ETD_ARB_0329254 successful
SYM_ARB_0656632 successful
RYD_ARB_0767889 successful
YM7_ARB_0274784 successful
ETD_ARB_0263528 successful
ETD_ARB_0018018 successful
RYD_ARB_0432060 successful
TRN_ARB_0087247 successful
YM7_ARB_0231365 successful
YM7_ARB_0239193 successful
SYM_ARB_0181059 successful
MSY_ARB_0204666 successful
MSY_ARB_0035173 successful
MTL_ARB_0089439 successful
TRN_ARB_0045423 successful
RYD_ARB_0815812 successful
SYM_ARB_0218752 successful
M

MTL_ARB_0070555 successful
TRN_ARB_0143685 successful
SHG_ARB_0053269 successful
MTL_ARB_0007439 successful
YM7_ARB_0190856 successful
YM7_ARB_0872577 successful
YM7_ARB_0880547 successful
MSY_ARB_0105636 successful
TRN_ARB_0015964 successful
SYM_ARB_0733660 successful
QBS_ARB_0803423 successful
SYM_ARB_0101793 successful
MTL_ARB_0094342 successful
QBS_ARB_0082202 successful
RYD_ARB_0420569 successful
MTL_ARB_0099472 successful
ETD_ARB_0277656 successful
SYM_ARB_0209964 successful
YM7_ARB_0465195 successful
MSY_ARB_0290512 successful
TRN_ARB_0164598 successful
YM7_ARB_0166575 successful
ETD_ARB_0125008 successful
TRN_ARB_0080683 successful
MSY_ARB_0019999 successful
ETD_ARB_0126962 successful
MSY_ARB_0006071 successful
RYD_ARB_0133478 successful
MTL_ARB_0231163 successful
MSY_ARB_0045520 successful
RYD_ARB_0000536 successful
MTL_ARB_0277701 successful
TRN_ARB_0061999 successful
YM7_ARB_0250634 successful
YM7_ARB_0096517 successful
MTL_ARB_0220633 successful
ETD_ARB_0074313 successful
R

MTL_ARB_0316036 successful
RYD_ARB_0766596 successful
RYD_ARB_0846529 successful
ETD_ARB_0034884 successful
RYD_ARB_0726098 successful
ETD_ARB_0318378 successful
QBS_ARB_0218781 successful
QBS_ARB_0014118 successful
ETD_ARB_0302863 successful
YM7_ARB_0338885 successful
MSY_ARB_0183703 successful
QBS_ARB_0753871 successful
SYM_ARB_0597511 successful
YM7_ARB_0536741 successful
SYM_ARB_0421726 successful
YM7_ARB_0651448 successful
TRN_ARB_0144509 successful
MSY_ARB_0096480 successful
YM7_ARB_0132950 successful
ETD_ARB_0094437 successful
YM7_ARB_0313373 successful
ETD_ARB_0176982 successful
MTL_ARB_0434777 successful
YM7_ARB_0486593 successful
RYD_ARB_0739746 successful
QBS_ARB_0171919 successful
QBS_ARB_0058141 successful
MSY_ARB_0001544 successful
QBS_ARB_0249840 successful
MTL_ARB_0443605 successful
RYD_ARB_0385422 successful
MSY_ARB_0097011 successful
SYM_ARB_0764962 successful
TRN_ARB_0121627 successful
QBS_ARB_0127538 successful
YM7_ARB_0527948 successful
YM7_ARB_0203674 successful
Y

...

QBS_ARB_0009032 successful
SYM_ARB_0166663 successful
YM7_ARB_1007473 successful
ETD_ARB_0051867 successful
SYM_ARB_0164698 successful
SYM_ARB_0358305 successful
TRN_ARB_0078897 successful
TRN_ARB_0136644 successful
YM7_ARB_0198098 successful
SHG_ARB_0107925 successful
RYD_ARB_0420118 successful
MSY_ARB_0005492 successful
SHG_ARB_0105221 successful
SYM_ARB_0585455 successful
YM7_ARB_0458998 successful
ETD_ARB_0046114 successful
SYM_ARB_0140361 successful
QBS_ARB_0755903 successful
MSY_ARB_0035937 successful
RYD_ARB_0762241 successful
MTL_ARB_0084192 successful
SYM_ARB_0216951 successful
SYM_ARB_0770195 successful
YM7_ARB_0643989 successful
MTL_ARB_0048130 successful
MTL_ARB_0397768 successful
SYM_ARB_0431762 successful
YM7_ARB_0872289 successful
YM7_ARB_0560030 successful
TRN_ARB_0239449 successful
MSY_ARB_0079487 successful
ETD_ARB_0300775 successful
YM7_ARB_0820541 successful
YM7_ARB_0696160 successful
TRN_ARB_0235816 successful
YM7_ARB_0188410 successful
ETD_ARB_0202869 successful
E

YM7_ARB_0107142 successful
MTL_ARB_0281905 successful
RYD_ARB_0822182 successful
RYD_ARB_0384604 successful
TRN_ARB_0253818 successful
MTL_ARB_0105487 successful
RYD_ARB_0384724 successful
MSY_ARB_0139997 successful
RYD_ARB_0038902 successful
YM7_ARB_0182990 successful
SYM_ARB_0593877 successful
RYD_ARB_0722384 successful
RYD_ARB_0365099 successful
TRN_ARB_0122437 successful
SYM_ARB_0687586 successful
YM7_ARB_0562672 successful
YM7_ARB_0411920 successful
MSY_ARB_0228014 successful
TRN_ARB_0269730 successful
ETD_ARB_0244119 successful
MSY_ARB_0079890 successful
QBS_ARB_0116857 successful
SYM_ARB_0264425 successful
SHG_ARB_0026454 successful
QBS_ARB_0041433 successful
SHG_ARB_0027568 successful
RYD_ARB_0855422 successful
QBS_ARB_0110584 successful
ETD_ARB_0246398 successful
YM7_ARB_0362406 successful
YM7_ARB_0000207 successful
SYM_ARB_0019319 successful
MSY_ARB_0130603 successful
SYM_ARB_0258239 successful
RYD_ARB_0351646 successful
SYM_ARB_0824972 successful
RYD_ARB_0783460 successful
M

SYM_ARB_0683978 successful
QBS_ARB_0238939 successful
YM7_ARB_0219742 successful
RYD_ARB_0060686 successful
YM7_ARB_0385567 successful
RYD_ARB_0081316 successful
MSY_ARB_0235768 successful
YM7_ARB_0568643 successful
MSY_ARB_0197413 successful
SYM_ARB_0657961 successful
SYM_ARB_0376463 successful
TRN_ARB_0267983 successful
YM7_ARB_0001511 successful
RYD_ARB_0001038 successful
RYD_ARB_0310303 successful
MSY_ARB_0258219 successful
YM7_ARB_0375472 successful
RYD_ARB_0120229 successful
RYD_ARB_0724210 successful
RYD_ARB_0065147 successful
SYM_ARB_0304996 successful
MTL_ARB_0439117 successful
MTL_ARB_0253771 successful
SYM_ARB_0207125 successful
YM7_ARB_0030621 successful
QBS_ARB_0728410 successful
ETD_ARB_0318171 successful
YM7_ARB_0290494 successful
TRN_ARB_0144698 successful
TRN_ARB_0079461 successful
MTL_ARB_0357643 successful
SYM_ARB_0025485 successful
YM7_ARB_0740950 successful
MTL_ARB_0417957 successful
SBN_ARB_0058277 successful
YM7_ARB_0204033 successful
YM7_ARB_0826884 successful
S

MTL_ARB_0130587 successful
MSY_ARB_0085029 successful
ETD_ARB_0276320 successful
SYM_ARB_0552350 successful
QBS_ARB_0109820 successful
MTL_ARB_0242648 successful
YM7_ARB_0768086 successful
SYM_ARB_0521115 successful
MSY_ARB_0186603 successful
YM7_ARB_0204569 successful
YM7_ARB_0776088 successful
ETD_ARB_0139882 successful
RYD_ARB_0419809 successful
RYD_ARB_0759529 successful
YM7_ARB_0263778 successful
SYM_ARB_0263274 successful
TRN_ARB_0262898 successful
MSY_ARB_0173433 successful
RYD_ARB_0107521 successful
RYD_ARB_0315377 successful
YM7_ARB_0662116 successful
YM7_ARB_0647242 successful
YM7_ARB_0339074 successful
YM7_ARB_0826089 successful
SYM_ARB_0283722 successful
SHG_ARB_0097072 successful
MTL_ARB_0311879 successful
YM7_ARB_0812389 successful
ETD_ARB_0304854 successful
RYD_ARB_0758875 successful
TRN_ARB_0110972 successful
SYM_ARB_0710164 successful
ETD_ARB_0118652 successful
YM7_ARB_0989122 successful
MTL_ARB_0346306 successful
RYD_ARB_0072139 successful
MTL_ARB_0401871 successful
M

/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/date_fixer.py:276: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dateline'] = pd.to_datetime(date_list)


In [ ]:
prefix= 'non_tagged_'

non_tagged_files.

for file in os.listdir(filepath):
    if prefix in file:
        



In [ ]:
for val in copied.tags.values:
    if len(val) > 1:
        print(val)

In [ ]:
sub['america'] = sub.apply(lambda x: check_for_america(x.en_text), axis=1)
sub.head(15)

In [ ]:
sub[sub.america==True].label.value_counts()

In [ ]:
pd.read_csv(publication_names[4] + '.csv')

In [ ]:
csvs = []
for name in publication_names:
    csvs.append(pd.read_csv(name + '.csv'))
df = pd.concat(csvs)
df.to_csv('all_articles.csv', index=False)

In [ ]:
df.source.value_counts()

In [ ]:
df.shape

In [ ]:
df.sample(500).to_csv('sample_articles.csv', index=False)

In [ ]:
steps = int(df.shape[0] /3) 

In [ ]:
split_dfs = []
for i in range(0, df.shape[0] - steps, steps):
    split_dfs.append(df.iloc[i: i+steps])
    
split_dfs[0].text.unique().size
    

In [ ]:
print(i)

In [ ]:
for i, d in enumerate(split_dfs):
    d.to_csv(f'block_{i+1}.csv', index=False)

In [ ]:
split_dfs[2].head()

In [ ]:
summarizer = pipeline('summarization')

In [ ]:
def make_msa(df_text):
    try:
        done = msa(df_text)
        return done
    except:
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
                done = [{'label': label, 'score': score}]
                return done
        except:
            print(df_text)
            return False
        

def analyze_text(df):
    scores = []
    scores = df.iloc[0:50].text.apply(make_msa)
    #df['score'] = df.text.apply(lambda x: msa(x)[0]['score'] if len(x.split()) < 300 else False)
    return scores

def analyze_headline(df):
    headline_scores = []
    scores = df.iloc[0:200].headline.apply(make_msa)
    return scores

#(analyze_text(df) != False).sum()
(analyze_text(subset_df) != False).sum()

In [ ]:
text= 'ائيل عن وثائق جديدة اليوم، السبت، عن أسرار حرب الاستنزاف التى خاضتها مصر ض'

In [ ]:
subset_df = pd.read_csv('sample_articles.csv')
subset_df.head()

In [ ]:
df.iloc[0]

In [ ]:
for val in subset_df.text.values:
    if text in val:
        print(subset_df[subset_df.text==val])

In [ ]:
source_dicts = {}
for source in df.source.unique():
    source_dicts[source] = df[df.source == source]
    
source_dicts['SaudiYoum']
    

In [ ]:
import re

In [ ]:
df.dateline.values[0]

In [ ]:
for val in list(df.dateline.values):
    try:
        print(re.search('20\d\d\W\d\d\W\d\d', val).group())
        
    except:
        print(val)

#print(len(df.dateline.values))

In [ ]:
source_dicts['SaudiYoum'].dateline.values[0]

In [ ]:


def using_ahocorasick_months(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    return mask

def look_for_words_in_text_months(df_text):
    
    months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
    months = flip_key_value_pairs(months)
    
    tags = []
    for key in months.keys():
        if key in df_text:
            tags.append(key)
            
    return tags

mask = using_ahocorasick(df.dateline, list(months.values()))[0]
copy = df[mask].copy()
copy = pd.DataFrame(copy)
copy['month'] = copy.dateline.apply(look_for_words_in_text_months)


In [ ]:
copy

In [ ]:
def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False

df.dateline.apply(find_month)

In [ ]:
df.dateline.size

In [ ]:
df=pd.read_csv('all_articles.csv')

In [ ]:
import pandas as pd
from googletrans import Translator
from camel_tools.sentiment import SentimentAnalyzer
from transformers import pipeline


""" 
Change name to which file you're doing sentiment analysis on, NOT including the '.csv' 
Output file will be 'labeled_<filename>.csv'
"""

msa = pipeline('text-classification', model="CAMeL-Lab/bert-base-arabic-camelbert-msa-sentiment")



def load_and_label_df(name):
    path = 'split_articles/'
    df = load_csv(path + name)
    print(f'loaded {name}')
    print('labeling/scoring...')
    df = create_labels_scores(df, name)
    print('done labeling/scoring!')
    return df

def load_csv(filename):
    df = pd.read_csv(filename)
    return df

def make_msa(df_text):
    
    try:
        done = msa(df_text)
        return done
    except:
        
        try:
            first_half = msa(df_text[:round(len(df_text)/2)]) 
            second_half = msa(df_text[round(len(df_text)/2):])
            if first_half[0]['label'] == second_half[0]['label']:
                label = first_half[0]['label']
                score = (test_1[0]['score'] + test_2[0]['score'])/2
            done = [{'label': label, 'score': score}]
            return done
        except:
        
            try:
                beginning = msa(df_text[:round(len(df_text)/3)]) 
                middle = msa(df_text[round(len(df_text)/3):round(len(df_text)*2/3)])
                end = msa(df_text[round(len(df_text)*2/3):])

                if (beginning[0]['label'] == middle[0]['label']) and (beginning[0]['label'] == end[0]['label']):
                    label = first_half[0]['label']
                    score = (beginning[0]['score'] + middle[0]['score'] + end[0]['score'])/3
                    done = [{'label': label, 'score': score}]
                    return done
                else:
                    return 'sentiment of parts not equal'
            except:
                return '3 is not enough'
        
def analyze_text(df):
    scores = []
    print('analyzing_texts')
    scores = [make_msa(val) for val in df.text.values]
    return scores

def analyze_headline(df):
    print('analyzing headlines')
    headline_scores = [make_msa(val) for val in df.headline.values]
    return headline_scores

def label_and_scores(msa_scores):
    labels = []
    scores = []
    for val in msa_scores:
        try:
            labels.append(val[0]['label'])
            scores.append(val[0]['score'])
        except:
            labels.append(False)
            scores.append(False)

    return labels, scores

def create_labels_scores(df, name):
    text_scores = analyze_text(df)
    labels, scores = label_and_scores(text_scores)
    df['text_label'] = labels
    df['text_score'] = scores

    headline_scores = analyze_headline(df)
    labels, scores = label_and_scores(headline_scores)
    df['headline_label'] = labels
    df['headline_score'] = scores

    # CHANGE 'BLOCK_NAME' TO WHATEVER YOU WANT
    df.to_csv('labeled_split_articles/labeled_'+ name + '.csv', index=False)

    return df

#load_and_label_df(name)




In [ ]:
pd.read_csv('labeled_' +name+'.csv')

In [ ]:
def split_tons_of_csvs():
    df = pd.read_csv('/Users/kylegreen/codeup-data-science/all_articles.csv')
    
    end_point = '/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/split_articles/'
    
    df_shape = df.shape[0]
    
    one_thousandth = int(df_shape/1000)
    
    for i in range(0, df_shape, one_thousandth):
        copy = df.iloc[i:i+one_thousandth].copy()
        copy.to_csv(end_point+str(i) + '.csv', index=False)
    
split_tons_of_csvs().head()

In [ ]:
pd.read_csv('split_articles/354501.csv')

In [ ]:
import os
import pandas as pd


labeled = 'labeled_split_articles'



bunk_files = []

for fi in os.listdir(labeled):
    df = pd.read_csv(labeled + '/' + fi)
    if df.text_score.sum() == 0:
        bunk_files.append(fi)
        
for fi in bunk_files:
    if os.path.isfile(f'{labeled}/{fi}'):
        os.remove(f'{labeled}/{fi}')
    else:
        print(f"{fi} doesn't exist")
        

In [ ]:
import regex as re
import pandas as pd

def find_month(df_dateline):
    exp = '\d\d.*\d\d\d\d'
    try:
        month = re.search(exp, df_dateline).group()[2:-4]
        months = {'January' : 'يناير',
                        'February' : 'فبراير',
                        'March' : 'مارس',
                        'April' : 'أبريل',
                        'June' : 'يوني',
                        'July' : 'يولي',
                        'August' : 'أغسطس',
                        'September' : 'سبتمبر',
                        'October' : 'أكتوبر',
                        'November' : 'نوفمبر',
                        'December' : 'ديسمبر',
                        'April1' : 'إبريل',
                        'May': 'ماي'}
    
        months = flip_key_value_pairs(months)
        
        return re.search(exp, df_dateline).group()[2:-4]
    except:
        
        try:
            return re.search('20\d\d\W\d\d\W\d\d', val).group()

        except:
            return False


def flip_key_value_pairs(dicts):
    res = dict((v,k) for k,v in dicts.items())
    return res

df.head()

In [ ]:
pd.read_csv('labeled_split_articles/labeled_312732.csv.csv')

In [39]:
path = '/Users/kylegreen/codeup-data-science/arabic_media_nlp_project/kyles_work/labeled_split_articles'
name = 'labeled_187425.csv.csv'
filepath = path+'/'+name

pd.read_csv(filepath)


,Unnamed: 0,id,url,headline,dateline,text,tags,source,text_label,text_score,headline_label,headline_score
0,187425,SBN_ARB_0067749,http://www.sabanews.net/ar/news326168.htm,قائد الحرس الثوري الايراني: مكالمة روحاني الها...,30/سبتمبر/2013,30/سبتمبر/2013] طهران – سبأنت: اعتبر قائد قوات...,['أوباما'],Sabanews,neutral,0.9828380346298218,neutral,0.821420
1,187426,SBN_ARB_0067755,http://www.sabanews.net/ar/news326175.htm,قيادة وزارة الدفاع ورئاسة هيئة الأركان تنعيان ...,01/أكتوبر/2013,01/أكتوبر/2013] صنعاء - سبأنت: نعت قيادة وزار...,['القاعدة'],Sabanews,neutral,0.5289005637168884,neutral,0.856138
2,187427,SBN_ARB_0067760,http://www.sabanews.net/ar/news326181.htm,البيت الأبيض يأمر ببدء إغلاق وكالات حكومية,01/أكتوبر/2013,01/أكتوبر/2013] واشنطن - سبأنت : أمر البيت الأ...,['واشنطن'],Sabanews,neutral,0.9577153921127319,neutral,0.758667
3,187428,SBN_ARB_0067761,http://www.sabanews.net/ar/news326182.htm,ايفرتون يقفز الى المركز الرابع بالدوري الإنكليزي,01/أكتوبر/2013,01/أكتوبر/2013] لندن - سبأنت : واصل ايفرتون عر...,['بوش'],Sabanews,neutral,0.9606504440307617,neutral,0.887133
4,187429,SBN_ARB_0067781,http://www.sabanews.net/ar/news326204.htm,تشييع جثمان شهيد الواجب الجندي هلال على محمد ا...,01/أكتوبر/2013,01/أكتوبر/2013] صنعاء – سبأنت: شيع اليوم بصنعا...,['القاعدة'],Sabanews,neutral,0.6841258406639099,neutral,0.940607
...,...,...,...,...,...,...,...,...,...,...,...,...
352,187777,SBN_ARB_0073151,http://www.sabanews.net/ar/news332352.htm,مارادونا : اتحاد (الفيفا) ظلمني حين استبعدني م...,25/نوفمبر/2013,25/نوفمبر/2013] دبي ـ سبأنت: اكد أسطورة كرة ال...,['أمريكا'],Sabanews,neutral,0.7990878224372864,negative,0.922507
353,187778,SBN_ARB_0073153,http://www.sabanews.net/ar/news332355.htm,هولندا تستعد لاستضافة قمة دولية عن الأمن النوو...,25/نوفمبر/2013,25/نوفمبر/2013] لاهاي ـ سبأنت: تستضيف هولندا ف...,['واشنطن'],Sabanews,neutral,0.9840840697288513,neutral,0.985211
354,187779,SBN_ARB_0073208,http://www.sabanews.net/ar/news332423.htm,فعالية في عدن للاحتفاء باليومين العالمين للطفل...,26/نوفمبر/2013,26/نوفمبر/2013] عدن - سبأنت: نظمت الجمعية اليم...,['بوش'],Sabanews,neutral,0.9391960501670837,neutral,0.888983
355,187780,SBN_ARB_0073277,http://www.sabanews.net/ar/news332494.htm,تنفيذ مشاريع للمياه والصرف الصحي بمديرتي زنجبا...,26/نوفمبر/2013,26/نوفمبر/2013] أبين ـ سبأنت: يجري العمل حاليا...,['القاعدة'],Sabanews,neutral,0.9197974801063538,neutral,0.944486


In [40]:
def search_words_in_text(col, lst):
    A = ahocorasick.Automaton(ahocorasick.STORE_INTS)
    for word in lst:
        A.add_word(word.lower())
    A.make_automaton() 
    col = col.astype(str)
    col = col.str.lower()
    mask = col.apply(lambda x: bool(list(A.iter(x))))
    tags = col.apply(lambda x: list(A.iter(x)))
    return mask, tags

In [12]:
features = '''
35,source_Techreen,8943.448879256535
42,country_Syria,8943.448879256423
28,source_Almasryalyoum,2355.0164748339
10,America,1749.3620117708529
43,country_UAE,1254.1639088750942
27,source_Alittihad,1254.1639088750908
36,source_Youm7,1254.1038226182388
23,scaled_date,1028.351322645141
21,Bush,825.5788927612839
34,source_SaudiYoum,563.4215943320206
44,country_Yemen,532.3199952120876
33,source_Sabanews,532.3199952120876
41,country_Saudi_Arabia,359.4754705031025
1,capt_bag,272.2506389745735
7,Google,239.54317229043158
26,ownership_status,223.20219347351065
30,source_Alqabas,176.74692556317044
39,country_Kuwait,176.74692556317044
6,Gulf War,130.54652729478855
9,Cheney,124.84083618600735
2,nick,123.55177378850968
0,sept_11,89.44799347238539
15,Al Qaida,51.68456312757608
5,9/11,39.471061162490344
19,Facebook,36.54992633143914
22,is_ramadan,32.03118863502156
11,Obama,23.55767713271109
40,country_Lebanon,23.42292060787511
29,source_Almustaqbal,23.42292060787511
17,American,22.758160094685515
16,American (f),21.55333870326934
18,Osama Bin Laden,18.60268002713719
13,September 11,17.2437243222949
20,Clinton,10.634466768561827
31,source_Echoroukonline,9.728960860796375
37,country_Algeria,9.728960860796354
38,country_Egypt,9.723456348417018
24,source_start_year,8.995572219180271
25,source_start_year_scaled,8.995572219063023
3,arab,4.302323138409529
8,Washington,1.9141755003131955
14,Al Gore,1.3483615626056054
32,source_Ryiadh,1.1138538406404375
4,World Trade Center,0.22061950331823033
12,American (pl),0.007503714906288203'''

features = features.split(',')
my_range= range(1,91, 2)

final_features = []
for i in my_range:
    final_features.append(features[i])
    
print(final_features[:21])

['source_Techreen', 'country_Syria', 'source_Almasryalyoum', 'America', 'country_UAE', 'source_Alittihad', 'source_Youm7', 'scaled_date', 'Bush', 'source_SaudiYoum', 'country_Yemen', 'source_Sabanews', 'country_Saudi_Arabia', 'capt_bag', 'Google', 'ownership_status', 'source_Alqabas', 'country_Kuwait', 'Gulf War', 'Cheney', 'nick']


In [17]:
df = pd.read_csv('vars_with_years.csv').drop(columns='Unnamed: 0')
df

,sept_11,capt_bag,nick,arab,Osama Bin Laden,Clinton,Cheney,9/11,Bush,Facebook,...,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,text_label
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
305307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
305308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0
305309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [15]:
df.columns

Index(['Unnamed: 0', 'sept_11', 'capt_bag', 'nick', 'arab', 'Osama Bin Laden',
       'Clinton', 'Cheney', '9/11', 'Bush', 'Facebook', 'Washington',
       'American (f)', 'Google', 'America', 'American', 'American (pl)',
       'World Trade Center', 'Gulf War', 'September 11', 'Al Gore', 'Al Qaida',
       'Obama', 'is_ramadan', 'source_start_year', 'scaled_date',
       'source_start_year_scaled', 'ownership_status', 'source_Alittihad',
       'source_Almasryalyoum', 'source_Almustaqbal', 'source_Alqabas',
       'source_Echoroukonline', 'source_Ryiadh', 'source_Sabanews',
       'source_SaudiYoum', 'source_Techreen', 'source_Youm7',
       'country_Algeria', 'country_Egypt', 'country_Kuwait', 'country_Lebanon',
       'country_Saudi_Arabia', 'country_Syria', 'country_UAE',
       'country_Yemen'],
      dtype='object')